# Get Tweet Sentiment

In [ ]:
import json
import pandas as pd
import requests

config = json.load(open("config.json"))
text_analytics_sentiment_url = "https://westcentralus.api.cognitive.microsoft.com/text/analytics/v2.0/sentiment"

In [ ]:
df = pd.read_csv("./tweets.csv")
languages_df = pd.read_csv("./detected_languages.csv")

In [ ]:
df.head()

In [ ]:
languages_df.head()

In [ ]:
languages_df.language.unique()

In [ ]:
df = pd.merge(df, languages_df, left_index=True, right_index=True)
df.head()

In [ ]:
documents = {"documents": []}

for idx, row in df.iterrows():
    documents["documents"].append({
        "id": str(idx + 1),
        "language": row["language"],
        "text": row["text"]
    })

In [ ]:
documents

In [ ]:
headers = {"Ocp-Apim-Subscription-Key": config["subscriptionKey"]}
response  = requests.post(text_analytics_sentiment_url, headers=headers, json=documents)
sentiments = response.json()

In [ ]:
sentiments

In [ ]:
sentiment_df = pd.DataFrame([d["score"] for d in sentiments["documents"]], index=[d["id"] for d in sentiments["documents"]], 
             columns=["sentiment_score"])
sentiment_df.head()

In [ ]:
sentiment_df["sentiment_percentage"] = round(sentiment_df.sentiment_score * 100, 2)
sentiment_df.head()

In [ ]:
sentiment_df.sentiment_percentage.describe()

In [ ]:
sentiment_df[sentiment_df.sentiment_percentage == sentiment_df.sentiment_percentage.max()]

In [ ]:
[doc for doc in documents["documents"] if doc["id"] == "20"]

In [ ]:
min_sentiment = sentiment_df[sentiment_df.sentiment_percentage == sentiment_df.sentiment_percentage.min()]
min_sentiment

In [ ]:
[doc for doc in documents["documents"] if doc["id"] in min_sentiment.index.values]

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()
%matplotlib inline

g = sns.countplot(sentiment_df.sentiment_percentage)
loc, labels = plt.xticks()
g.set_xticklabels(labels, rotation=90)
g.set_ylabel("Count")
g.set_xlabel("Sentiment %")